# DEPENDENCIES

-------------------------------------------------------------------------------------------------------------------

In [ ]:
#!/usr/bin/env python3
# coding: utf-8

# color pallet greens https://coolors.co/34463e-a4c3b2-cce3de-eaf4f4-f6fff8
# color pallet secondary https://coolors.co/353535-3c6e71-ffffff-d9d9d9-284b63

import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import os

#Orca must be installed?
#conda install -c plotly plotly-orca
#Kaleido is for exporting images from plotly
#pip install -U kaleido

# DATA

-------------------------------------------------------------------------------------------------------------------

In [ ]:
data_df = pd.read_csv('../data_local/organized.csv')
col_names = []
for col in data_df.columns:
    col_names.append(col)
col_data = []
for i in col_names:
    col_data.append(data_df[i])

In [ ]:
data_df = data_df.drop(columns = ['date_p', 'account', 'balance'])
#data_df.iloc[[300]]

# STACKED BAR GRAPH

-------------------------------------------------------------------------------------------------------------------

In [ ]:
# pos_amount = [i*-1 for i in data_df['amount']]
# import plotly.express as px
# #wide_df = px.data.medals_wide()
# fig = px.bar(data_df,
#              x="date", y= data_df['category'],
#              color = pos_amount,
#              title="Total Spending by Category by Day")
# fig.show()

In [ ]:
type(data_df['amount'][0])

In [ ]:
#categories = data_df['category'].unique()
# sum_amount = data_df.loc[data_df['category'] == 'food']
# sum_amount['amount'].sum().round(2)

In [ ]:
# Need to make a DF where all the unique categories are summed

In [ ]:
sum_categories = data_df.groupby(['category', 'date']).sum().sort_values("date")
# sum_categories = sum_categories.rename(columns={'category':'categories'})
sum_categories

In [ ]:
#sum_categories['categories'] = sum_categories.index
sum_categories.reset_index(inplace = True)
sum_categories['amount'] = [i*-1 for i in sum_categories['amount']]
sum_categories = sum_categories[['category', 'amount']]
sum_categories

In [ ]:
cats = sorted(list(sum_categories.category.unique()))
cats.remove('payment')
cats

In [ ]:
summed_cats = []
for i in cats:
    summed_cats.append(int(round(sum_categories.loc[sum_categories['category'] == i].amount.sum())))
#summed_cats

In [ ]:
total_spending = sum(summed_cats)
total_spending
percent = []
for i in summed_cats:
    i = i * -1
    percent.append(round(i/(total_spending*-1)*100))
percent

In [ ]:
sum_totals = pd.DataFrame()
sum_totals['categories'] = cats
sum_totals['amounts'] = summed_cats
sum_totals['percent'] = percent
sum_totals

In [ ]:
# import plotly.express as px
# #wide_df = px.data.medals_wide()
# fig = px.bar(sum_categories,
#              x="date", y= sum_categories['category'],
#              color = 'amount',
#              labels = {'category':'amount'},
#              title="Total Spending by Category by Day")
# fig.show()

In [ ]:
# import plotly.express as px
# #wide_df = px.data.medals_wide()
# fig = px.bar(sum_categories, x="date", y= sum_categories['amount'], color ='amount', title="Total Spending by Category by Day")
# fig.show()

# ORIGINAL STACKED BAR GRAPH

-------------------------------------------------------------------------------------------------------------------

In [ ]:
import plotly.express as px
wide_df = px.data.medals_wide()
fig = px.bar(wide_df, x="nation", y=["gold", "silver", "bronze"], title="Wide-Form Input")
fig.show()

# TABLES

-------------------------------------------------------------------------------------------------------------------

In [ ]:
def get_length(df):
    col = ''
    for i in df.columns:
        get_length = df[i].map(len).max()
    return get_length

In [ ]:
def convert_str(series):
    return [str(i) for i in series]

In [ ]:
#Somehow because this is a jupyter notebook this function reassigns the column data that is not a string,
# to string and keeps that value within the imported dataframe
def get_max(df):
    max_num = 0
    max_col = ''
    max_list = []
    for i in range(len(df.columns)):
        if df[df.columns[i]][0] == type('s'):
            continue
        else:
            df[df.columns[i]] = convert_str(df[df.columns[i]])
        get_length = df[df.columns[i]].map(len).max()
        max_list.append(get_length)
        if max_num < get_length:
            max_num = get_length
            max_col = df.columns[i]
            col_num = i
    return max_num, max_col, max_list

In [ ]:
def make_table(df, colors, rowOddColor, rowEvenColor):
    if not os.path.exists('images'):
        os.mkdir('images')
    fig = go.Figure(data=[go.Table(
        columnorder = [i+1 for i in range(len(df.columns))],
        columnwidth = [i*30 for i in get_max(df)[2]],
        header=dict(values=list(df.columns),
                    fill_color=colors[0],
                    align='left',
                    font = dict(family="Arial", size=14, color= colors[1])),
        cells=dict(values=[df.date, df.transaction, df.amount, df.identifier, df.category],
                   fill_color= [[rowOddColor, rowEvenColor, rowOddColor, rowEvenColor]*4],#colors[2],
                   align='left',
                   font = dict(family="Arial", size=12, color=colors[3])))])

    fig.write_image("images/table.png", format="png")#, width=600, height=350, scale=2)
    #pdf is higher quality however it omits all the lines below
    max_height = len(df) * 50
    pio.write_image(fig, 'images/table.pdf', width=700, height=max_height)
    fig.show()

In [ ]:
type(data_df['amount'][0])

In [ ]:
len(data_df)

In [ ]:
# colors ['fill_color_header/bkgd', 'font_color_header/light', 'cells_color/light', 'cells_font_color/dk']
rowEvenColor = '#cce3de'
rowOddColor = '#EAF4F4'
green_colors = ['#6b9080', '#f6fff8', '#cce3de', '#34463E']
make_table(data_df, green_colors, rowEvenColor, rowOddColor)

In [ ]:
# color pallet https://coolors.co/353535-3c6e71-ffffff-d9d9d9-284b63
# colors ['fill_color_header/bkgd', 'font_color_header/light', 'cells_color/light', 'cells_font_color/dk']
blue_colors = ['#284B63','#FFFFFF', '#D9D9D9', '#353535']
white = '#FFFFFF'
grey = '#D9D9D9'
make_table(data_df, blue_colors, grey, white)

In [ ]:
type(data_df['amount'][1])

# DICTIONARY OPTIONS

-------------------------------------------------------------------------------------------------------------------

In [ ]:
# Thinking about adding another column based on this data
# dataframe might look good like
# where parent = budget category
# | parent       | type         | priority     | fixed/variable | one-time/re-occuring | identifier    | amount  | date    | transaction                   |
# | health       |life insurance| essential    | fixed          | re-occuring          | 'AAA Insurance| -50     | 01/22/21| AAA INSURANCE PAYMENT 48684   |
# | entertainment|movies        | non-essential| variable       | one-time             | 'REGAL'       | -15     | 01/12/21| CINEMA REGAL PARK WEST VA 1588|

fixed = {
    'debt':['car loan', 'student loan/tution'],
    'housing':['home//renters insurance', 'rent//mortgage', 'property taxes'],
    'utilities':['phone', 'cable//internet', 'garbage'],
    'medical':['health insurance'],
    'childcare':['daycare', 'child support'],
    'entertainment':['streaming services']
}

essential = {
    'savings' : ['life insurance', 'emergency fund', 'ROTH IRA', '401(k)', 'college fund', 'mad money'],
    'debt' : ['credit card', 'car loan', 'student loan/tuition', 'pay day loan', 'other' ],
    'housing' : ['home//renters insurance', 'rent//mortgage', 'repair//maitenance', 'furnishings', 'property taxes', 'yard', 'cleaning supply', 'kitchen ware','tools'],
    'transportation' : ['car insurance','maintenance', 'gas' 'bus/subway/train/ferry fare'],
    'food' : ['groceries', 'dining out', 'fast food', 'delivery', 'alcohol'],
    'utilities' : ['phone', 'cable//internet', 'water', 'gas', 'electricity', 'garbage'], #oil, wood
    'medical' : ['health insurance','doctor', 'prescriptions', 'first aid supply'], #types of doctors, optometrist, dentist, etc
    'education' : ['supplies', 'fees', 'books'],
    'childcare' : ['daycare', 'camps', 'babysitter', 'child support'],
    'personal' : ['clothing', 'toiletries//grooming']
}

non_essential = {
    'food' :['dining out', 'fast food', 'delivery', 'alcohol'],
    'gifts' : ['birthday', 'anniversary', 'holiday', 'other'],
    'giving': ['tithing', 'charitable', 'other'],
    'pets': ['food', 'supplies', 'vet', 'prescriptions'],
    'entertainment': ['streaming services','games', 'movies', 'hobbies', 'vacation', 'books'],
    'personal': ['salon', 'gym', 'dry cleaning']
    
}

# SUNBURST CHART

-------------------------------------------------------------------------------------------------------------------

In [ ]:
inside = ['savings', 'savings', 'debt', 'housing', 'transportation', 'food', 'utilities', 'medical', 'education', 'childcare', 'personal']

## Working
- go back to coolers.co and grab names of colors make a dictionary with the name as the key and color as value

In [ ]:
# red to blue/purple
rainbow_colors = ['#F94144', '#f3722c', '#F9844A', '#f8961e', '#F9C74F', '#90BE6D', '#43AA8B', '#4D908E', '#277DA1', '#577590']

In [ ]:
outside = ['essential' for i in range(len(inside))]

In [ ]:
# blues
# https://coolors.co/05668d-028090-00a896-02c39a-f0f3bd
# rainbow
# https://coolors.co/f94144-f3722c-f8961e-f9844a-f9c74f-90be6d-43aa8b-4d908e-577590-277da1

In [ ]:
sum_totals.head()

In [ ]:
import random
def constrain_sum(n, total):
    # Return an randomly chosen list of n positive intergers summing to 100/total
    dividers = sorted(random.sample(range(1, total), n-1))
    return [a - b for a,b in zip(dividers + [total], [0]+dividers)]


In [ ]:
inside = list(sum_totals.categories.unique())

In [ ]:
# summed = data_df.groupby(['category', 'amount']).sum().reset_index()
# summed

In [ ]:
outside = ['essential' for i in range(len(inside))]

In [ ]:
nums = constrain_sum(len(inside), 100)

In [ ]:
nums = [int(i)*-1 for i in sum_totals.amounts]

In [ ]:
import random
import plotly.express as px
data = dict(
    character= inside,
    parent = outside,
    value= nums)

fig =px.sunburst(
    data,
    names='character',
    parents='parent',
    values='value',
    title = 'Expenses'
)
fig.show()

In [ ]:
nums = sum_totals.percent

In [ ]:
import random
import plotly.express as px
data = dict(
    character= inside,
    parent = outside,
    value= nums)

fig =px.sunburst(
    data,
    names='character',
    parents='parent',
    values='value',
    title = 'Expenses'
)
fig.show()

# TREEEMAP CHART

-------------------------------------------------------------------------------------------------------------------

In [ ]:
# Need new TreeMap code, this one is not so pretty

In [ ]:
essential.keys()

In [ ]:
groups = list(essential.keys())
priority = ['Essential' for i in range(len(groups))]
nums = constrain_sum(len(groups), 100)
percentages = [str(nums[i])+'%' for i in range(len(nums))]

In [ ]:
groups

In [ ]:
df = pd.DataFrame(
    {
        'priority': priority,
        'categories':groups,
        'values': nums,
        'percentages':percentages
                  })

In [ ]:
import squarify
import matplotlib.pyplot as plt
# plot it
labels = [df['categories'][i].capitalize()+(' \n'+ str(nums[i])+'%') for i in range(len(df['categories']))]

squarify.plot(sizes=df['values'], label=labels, alpha=.4, color= rainbow_colors[:len(groups)+1])
plt.axis('off')
plt.show()

# ALTERNATE TREEMAP

-------------------------------------------------------------------------------------------------------------------

- FIX Legend
- Change rainbow colors from a list to a dictionary and grab the correct values accordingly'
- ie if data 100-90 red, 90-80 orange, 80-70 yellow, 70-60 green, 60-50 blue, 50-40, purple and on until 10-0
- fix hover put something relevant

In [ ]:
groups_all = list(essential.keys()) + list(non_essential)
nums_all = constrain_sum(len(groups_all), 100)
priority_all = ['Essential' if x in essential.keys() else 'Non Essential' for x in groups_all]
percentages_all = [str(nums_all[i])+'%' for i in range(len(nums_all))]
# Create a data frame with fake data
# df = pd.DataFrame({'nb_people':[8,3,4,2],
#                    'group':["Housing\n 40%",
#                             "Utilities\n 10%",
#                             "Food\n 10%",
#                             "Entertainment\n 10%"]})
df_all = pd.DataFrame(
    {
        'priority': priority_all,
        'categories':groups_all,
        'values': nums_all,
        'percentages':percentages_all
                  })

In [ ]:
colors_all = rainbow_colors[:len(groups_all)+1]
counter = len(colors_all)
while counter < len(df_all):
    colors_all.append(rainbow_colors[len(df_all)-counter])
    counter += 1
colors_all

In [ ]:
print(sorted(df['values']))

In [ ]:
df_all

In [ ]:
sorted_df = df_all.sort_values(by=['values', 'categories']).reset_index(drop=True)
sorted_df

In [ ]:
import plotly.express as px
fig = px.treemap(df_all.sort_values(by=['values','categories']).reset_index(drop=True), path=['priority', 'categories', 'percentages'], 
                 values='values', 
                 color='values')
fig.update_traces(marker_colors= colors_all, selector=dict(type='treemap')) 
#fig.update_traces(pathbar_visible=False, selector=dict(type='treemap'))
fig.show()

In [ ]:
px.data.tips()

In [ ]:
import plotly.express as px
  
df = px.data.tips()
  
fig = px.treemap(df, path=['day', 'time', 'tip'], 
                 values='total_bill', 
                 color='total_bill')

fig.show()

In [ ]:
import plotly.express as px

for name in dir(px.data):
    if '__' not in name:
        print(name)

In [ ]:
print(px.data.tips())